# Installation (only required for the first run)

In [ ]:
!git clone --recursive https://github.com/joshpapermaster/nerf_pl.git

%cd /content/nerf_pl
!pip install -r requirements.txt

%cd /content/nerf_pl/torchsearchsorted
!pip install .

import os

Cloning into 'nerf_pl'...
remote: Enumerating objects: 1260, done.
remote: Counting objects: 100% (393/393), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1260 (delta 376), reused 368 (delta 368), pack-reused 867
Receiving objects: 100% (1260/1260), 23.15 MiB | 19.62 MiB/s, done.
Resolving deltas: 100% (810/810), done.
Submodule 'torchsearchsorted' (https://github.com/aliutkus/torchsearchsorted.git) registered for path 'torchsearchsorted'
Cloning into '/content/nerf_pl/torchsearchsorted'...
remote: Enumerating objects: 373, done.        
remote: Counting objects: 100% (216/216), done.        
remote: Compressing objects: 100% (146/146), done.        
remote: Total 373 (delta 102), reused 157 (delta 62), pack-reused 157        
Receiving objects: 100% (373/373), 67.12 KiB | 2.03 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Submodule path 'torchsearchsorted': checked out '1e0ffc3e0663ffda318b4e28348efd90313d08f3'
/content/nerf_pl
     |█████████████████████

/content/nerf_pl/torchsearchsorted
Processing /content/nerf_pl/torchsearchsorted
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for torchsearchsorted: filename=torchsearchsorted-1.1-cp37-cp37m-linux_x86_64.whl size=3208442 sha256=f2ac4874076714f3c602670cb1a2f5e771cc36c1921e371cbb88075e4c5780d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-a6iaactg/wheels/f0/75/06/3b90bb6a54c933890a1ab2be264db76042598a2cd1aa71a29b
Successfully built torchsearchsorted


# Mount your google drive (to access data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# Train! (depending on number of epochs, takes about 5~8 hours)

### model weights are saved to `ckpts/$EXP`
### training logs (loss/PSNR evolution) are saved to `logs/$EXP`

## Forward facing scene

In [ ]:
%cd /content/nerf_pl

import os
# set training configurations here
os.environ['ROOT_DIR'] = "/content/drive/My Drive/Colab Notebooks/nerf_llff_data/fern"
                         # directory containing the data
os.environ['IMG_W'] = "504" # image width (do not set too large)
os.environ['IMG_H'] = "378" # image height (do not set too large)
os.environ['NUM_EPOCHS'] = "30" # number of epochs to train (depending on how many images there are,
                                # 20~30 might be enough)
os.environ['EXP'] = "fern" # name of the experience (arbitrary)

!python train.py \
   --dataset_name llff \
   --root_dir "$ROOT_DIR" \
   --N_importance 64 --img_wh $IMG_W $IMG_H \
   --num_epochs $NUM_EPOCHS --batch_size 1024 \
   --optimizer adam --lr 5e-4 \
   --lr_scheduler cosine \
   --exp_name $EXP

/content/nerf_pl
INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
val image is /content/drive/My Drive/Colab Notebooks/nerf_llff_data/fern/images/IMG_4038.JPG
Epoch 1:   3% 113/3536 [00:21<10:44,  5.32it/s, loss=0.055, train_psnr=15.7, v_num=0]INFO:lightning:Detected KeyboardInterrupt, attempting graceful shutdown...
Epoch 1:   3% 113/3536 [00:21<10:51,  5.26it/s, loss=0.055, train_psnr=15.7, v_num=0]
INFO:lightning:

Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_train_start      	|  0.00033142     	|  0.00033142     
on_epoch_start      	|  0.00028379     	|  0.00028379     
get_train_batch     	|  0.0045304      	|  0.51647        
on_batch_start      	|  1.6378e-05     	|  0.0018671      
model_forward       	|  0.030654       	|  3.4946         
model_backward      	|  0.11           	|  12.54          
on_after_backward   	|  4.1044e-06     	|

## 360 inward-facing scene

In [ ]:
%cd /content/nerf_pl

# set training configurations here
os.environ['ROOT_DIR'] = "/content/drive/My Drive/Colab Notebooks/nerf_llff_data/fern"
                         # directory containing the data
os.environ['IMG_W'] = "504" # image width (do not set too large)
os.environ['IMG_H'] = "378" # image height (do not set too large)
os.environ['NUM_EPOCHS'] = "30" # number of epochs to train (depending on how many images there are,
                                # 20~30 might be enough)
os.environ['EXP'] = "exp" # name of the experience (arbitrary)

!python train.py \
   --dataset_name llff \
   --root_dir "$ROOT_DIR" \
   --N_importance 64 --img_wh $IMG_W $IMG_H \
   --spheric --use_disp \
   --num_epochs $NUM_EPOCHS --batch_size 1024 \
   --optimizer adam --lr 5e-4 \
   --lr_scheduler cosine \
   --exp_name $EXP

/content/nerf_pl
INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
Traceback (most recent call last):
  File "train.py", line 180, in <module>
    trainer.fit(system)
  File "/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py", line 730, in fit
    model.prepare_data()
  File "train.py", line 80, in prepare_data
    self.train_dataset = dataset(split='train', **kwargs)
  File "/content/nerf_pl/datasets/llff.py", line 173, in __init__
    self.read_meta()
  File "/content/nerf_pl/datasets/llff.py", line 235, in read_meta
    rays_o, rays_d = get_rays(self.directions, c2w) # both (h*w, 3)
  File "/content/nerf_pl/datasets/ray_utils.py", line 43, in get_rays
    rays_d = rays_d / torch.norm(rays_d, dim=-1, keepdim=True)
  File "/usr/local/lib/python3.7/dist-packages/torch/functional.py", line 755, in norm
    return torch._C._VariableFunctions.frobenius_norm(input, dim, keepdim=keepdim)
KeyboardInterrupt


# Testing! (takes about 20~30 minutes)

### You can also download the pretrained model for `fern` [here](https://github.com/kwea123/nerf_pl/releases)

### The results are saved to `results/llff/$SCENE`

In [ ]:
os.environ['SCENE'] = 'fern'
os.environ['CKPT_PATH'] = "/content/fern.ckpt"

!python eval.py \
   --root_dir "$ROOT_DIR" \
   --dataset_name llff --scene_name $SCENE \
   --img_wh $IMG_W $IMG_H --N_importance 64 --ckpt_path $CKPT_PATH

100% 120/120 [19:19<00:00,  9.66s/it]


In [ ]:
ls

ckpts/                 LICENSE        opt.py            test.ipynb
datasets/              LICENSE_bmild  __pycache__/      torchsearchsorted/
docs/                  logs/          README.md         train.py
eval.py                losses.py      README_mesh.md    utils/
extract_color_mesh.py  metrics.py     README_Unity.md
extract_mesh.ipynb     models/        requirements.txt


In [ ]:
ls ../

drive/  nerf_pl/  sample_data/
